In [1]:
1+1

2

In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.pregel import NodeBuilder

load_dotenv()


class ChatState(TypedDict):
    question: str
    answer: str


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, verbose=True)

# 인메모리 캐쉬를 위해 MemorySaver를 사용합니다.
memory = MemorySaver()


# 노드 정의
def generate_answer(state: ChatState) -> dict:
    """LLM을 호출하여 답변을 생성하는 노드 함수"""
    print("\n>>>>> LLM 노드 실행! (이 메시지는 캐쉬되지 않았을 때만 보입니다) <<<<<")
    question = state["question"]
    response = llm.invoke(question)
    return {"answer": response.content}


# 캐쉬 함수 정의
def get_cache_key(state: ChatState, config: dict) -> str:
    return state["question"]


# 노드 빌더를 이용하여 Cache 기능을 갖는 노드를 정의함
cached_node_builder = NodeBuilder(generate_answer).cache(get_cache_key)

# StateGraph 빌더를 생성합니다.
builder = StateGraph(ChatState)

# NodeBuilder로 생성한 캐쉬 적용 노드를 그래프에 추가합니다.
builder.add_node("generate_answer_node", cached_node_builder.build())
builder.add_edge(START, "generate_answer_node")
builder.add_edge("generate_answer_node", END)

# Checkpointer(캐쉬)를 연결하여 그래프를 컴파일합니다.
graph = builder.compile(checkpointer=memory)






TypeError: NodeBuilder.__init__() takes 1 positional argument but 2 were given

In [ ]:
# ---  그래프 실행 및 캐싱 확인 ---
# 대화 스레드를 식별하기 위한 설정
config = {"configurable": {"thread_id": "chat-thread-1"}}
question = "LangGraph에서 노드 캐싱은 어떻게 하나요?"
initial_state = {"question": question, "answer": ""}

# 첫 번째 호출: LLM 노드가 실행됩니다.
print("--- 첫 번째 질문 ---")
print(f"질문: {question}")
final_state = graph.invoke(initial_state, config)
print(f"답변: {final_state['answer']}")

# 두 번째 호출: 동일한 질문이므로 캐쉬된 결과를 반환합니다.
print("\n\n--- 두 번째 질문 (동일한 내용) ---")
print(f"질문: {question}")
# 'LLM 노드 실행!' 메시지가 출력되지 않아야 합니다.
cached_state = graph.invoke(initial_state, config)
print(f"답변: {cached_state['answer']}")

# 세 번째 호출: 다른 질문이므로 LLM 노드가 다시 실행됩니다.
print("\n\n--- 세 번째 질문 (다른 내용) ---")
new_question = "LangChain의 장점은 무엇인가요?"
new_initial_state = {"question": new_question, "answer": ""}
print(f"질문: {new_question}")
new_final_state = graph.invoke(new_initial_state, config)
print(f"답변: {new_final_state['answer']}")